# 판매 중인 모델 number
- 승용
	- 더 뉴 캐스퍼 : 4671
	- 캐스퍼 일렉트릭 : 4653
	- 아반떼 : 4455
	- 아반떼 N : 4564
	- 쏘나타 디 엣지 : 4466
	- 더 뉴 아이오닉 6 : 4746
	- G70 : 3995
	- 그랜저 : 4188
	- G80 : 4603
	- Electrified G80 : 4660
	- G90 : 4016

- SUV
	- 베뉴 : 3654
	- 코나 : 4361
	- 코나 일렉트릭 : 4510
	- 아이오닉 5 : 4624
	- 아이오닉 5 N : 4558
	- New GV60 : 4701
	- 투싼 : 4592
	- 디 올 뉴 넥쏘 : 4677
	- GV70 : 4609
	- Electrified GV70 : 4705
	- 산타페 : 4435
	- 디 올 뉴 팰리세이드 : 4699
	- GV80 : 4465
	- 아이오닉 9 : 4088
	- 스타리아 : 4014

- 상용
	- 포터2 : 1901
	- 포터2 일렉트릭 : 4399
	- ST1 : 4626
	- 쏠라티 : 3297
	- 카운티 : 3502
	- 카운티 일렉트릭 : 4400
	- 마이티 : 3500
	- 파비스 : 4003

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time


# 브라우저 옵션 설정
options = Options()
options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# 웹 드라이버 실행

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 다나와 아반떼 CN7 제원 페이지 접속
url = "https://auto.danawa.com/auto/?Work=model&Model=4592&Tab=spec"
driver.get(url)

# 렌더링 대기
time.sleep(3)

# 렌더링된 HTML 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 1. 트림명 추출
tables = soup.find_all("table", class_="compare__table compare__header")
header_table = tables[-1]
trim_names = [span.get_text(strip=True) for span in header_table.find_all("span", class_="trim")]

# 2. 좌측 제원 항목명 추출 (id: compareLeft_XX)
spec_rows = soup.find_all("tr", class_="groupType_5 leftTr")
spec_names = []
spec_ids = []
for tr in spec_rows:
    if tr.get("id", "").startswith("compareLeft_"):
        spec_ids.append(tr["id"].split("_")[1])
        spec_names.append(tr.td.text.strip())

# 3. 트림별 제원값 추출 (id: compareRight_XX)
spec_values = []
for spec_id in spec_ids:
    tr = soup.find("tr", id=f"compareRight_{spec_id}")
    if tr:
        tds = tr.find_all("td")
        row_vals = [td.get_text(strip=True) for td in tds]
        spec_values.append(row_vals)
    else:
        spec_values.append([""] * len(trim_names))  # 값 없으면 빈값

# 4. DataFrame으로 정리
df = pd.DataFrame(spec_values, columns=trim_names, index=spec_names)
df.index.name = "항목"
df.reset_index(inplace=True)
df.head()

,항목,모던 2WD (A/T),모던 4WD (A/T),프리미엄 2WD (A/T),프리미엄 4WD (A/T),N Line 프리미엄 2WD (A/T),N Line 프리미엄 4WD (A/T),인스퍼레이션 2WD (A/T),인스퍼레이션 4WD (A/T),N Line 인스퍼레이션 2WD (A/T),N Line 인스퍼레이션 4WD (A/T)
0,전장,"4,640 mm","4,640 mm","4,640 mm","4,640 mm","4,650 mm","4,650 mm","4,640 mm","4,640 mm","4,650 mm","4,650 mm"
1,전폭,"1,865 mm","1,865 mm","1,865 mm","1,865 mm","1,865 mm","1,865 mm","1,865 mm","1,865 mm","1,865 mm","1,865 mm"
2,전고,"1,665 mm","1,665 mm","1,665 mm","1,665 mm","1,665 mm","1,665 mm","1,665 mm","1,665 mm","1,665 mm","1,665 mm"
3,축거,"2,755 mm","2,755 mm","2,755 mm","2,755 mm","2,755 mm","2,755 mm","2,755 mm","2,755 mm","2,755 mm","2,755 mm"
4,윤거 (전),"1,620 mm","1,620 mm","1,615 mm","1,615 mm","1,615 mm","1,615 mm","1,615 mm","1,615 mm","1,615 mm","1,615 mm"


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# ✔️ 크롤링 대상 모델 번호와 이름 (딕셔너리 형태로 관리)
car_models = {
    "더 뉴 캐스퍼": 4671,
    "캐스퍼 일렉트릭": 4653,
    "아반떼": 4455,
    "아반떼 N": 4564,
    "쏘나타 디 엣지": 4466,
    "더 뉴 아이오닉 6": 4746,
    "G70": 3995,
    "그랜저": 4188,
    "G80": 4603,
    "Electrified G80": 4660,
    "G90": 4016,
    "베뉴": 3654,
    "코나": 4361,
    "코나 일렉트릭": 4510,
    "아이오닉 5": 4624,
    "아이오닉 5 N": 4558,
    "New GV60": 4701,
    "투싼": 4592,
    "디 올 뉴 넥쏘": 4677,
    "GV70": 4609,
    "Electrified GV70": 4705,
    "산타페": 4435,
    "디 올 뉴 팰리세이드": 4699,
    "GV80": 4465,
    "아이오닉 9": 4088,
    "스타리아": 4014,
    "포터2": 1901,
    "포터2 일렉트릭": 4399,
    "ST1": 4626,
    "쏠라티": 3297,
    "카운티": 3502,
    "카운티 일렉트릭": 4400,
    "마이티": 3500,
    "파비스": 4003,
}

# 셀레니움 옵션
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 저장 폴더 생성
os.makedirs("car_specs", exist_ok=True)

# 모델 반복 크롤링
for car_name, model_number in car_models.items():
    print(f"🚗 크롤링 중: {car_name} ({model_number})")

    url = f"https://auto.danawa.com/auto/?Work=model&Model={model_number}&Tab=spec"
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser") # 렌더링된 HTML 파싱

    try:
		# 1. 트림명 추출
        tables = soup.find_all("table", class_="compare__table compare__header")
        header_table = tables[-1]
        trim_names = [span.get_text(strip=True) for span in header_table.find_all("span", class_="trim")]

		# 2. 좌측 제원 항목명 추출 (id: compareLeft_XX)
        spec_rows = soup.find_all("tr", class_="groupType_5 leftTr")
        spec_names, spec_ids = [], []
        for tr in spec_rows:
            if tr.get("id", "").startswith("compareLeft_"):
                spec_ids.append(tr["id"].split("_")[1])
                spec_names.append(tr.td.text.strip())
				
		# 3. 트림별 제원값 추출 (id: compareRight_XX)
        spec_values = []
        for spec_id in spec_ids:
            tr = soup.find("tr", id=f"compareRight_{spec_id}")
            if tr:
                tds = tr.find_all("td")
                row_vals = [td.get_text(strip=True) for td in tds]
                spec_values.append(row_vals)
            else:
                spec_values.append([""] * len(trim_names))

		# 4. DataFrame으로 정리
        df = pd.DataFrame(spec_values, columns=trim_names, index=spec_names)
        df.index.name = "항목"
        df.reset_index(inplace=True)

        # CSV 저장
        df.to_csv(f"car_specs/{car_name}.csv", index=False, encoding="utf-8-sig")
        print(f"✅ 저장 완료: {car_name}.csv")
    
    except Exception as e:
        print(f"❌ {car_name} 크롤링 실패: {e}")

driver.quit()


🚗 크롤링 중: 더 뉴 캐스퍼 (4671)
✅ 저장 완료: 더 뉴 캐스퍼.csv
🚗 크롤링 중: 캐스퍼 일렉트릭 (4653)
✅ 저장 완료: 캐스퍼 일렉트릭.csv
🚗 크롤링 중: 아반떼 (4455)
✅ 저장 완료: 아반떼.csv
🚗 크롤링 중: 아반떼 N (4564)
✅ 저장 완료: 아반떼 N.csv
🚗 크롤링 중: 쏘나타 디 엣지 (4466)
✅ 저장 완료: 쏘나타 디 엣지.csv
🚗 크롤링 중: 더 뉴 아이오닉 6 (4746)
✅ 저장 완료: 더 뉴 아이오닉 6.csv
🚗 크롤링 중: G70 (3995)
✅ 저장 완료: G70.csv
🚗 크롤링 중: 그랜저 (4188)
✅ 저장 완료: 그랜저.csv
🚗 크롤링 중: G80 (4603)
✅ 저장 완료: G80.csv
🚗 크롤링 중: Electrified G80 (4660)
✅ 저장 완료: Electrified G80.csv
🚗 크롤링 중: G90 (4016)
✅ 저장 완료: G90.csv
🚗 크롤링 중: 베뉴 (3654)
✅ 저장 완료: 베뉴.csv
🚗 크롤링 중: 코나 (4361)
✅ 저장 완료: 코나.csv
🚗 크롤링 중: 코나 일렉트릭 (4510)
✅ 저장 완료: 코나 일렉트릭.csv
🚗 크롤링 중: 아이오닉 5 (4624)
✅ 저장 완료: 아이오닉 5.csv
🚗 크롤링 중: 아이오닉 5 N (4558)
✅ 저장 완료: 아이오닉 5 N.csv
🚗 크롤링 중: New GV60 (4701)
✅ 저장 완료: New GV60.csv
🚗 크롤링 중: 투싼 (4592)
✅ 저장 완료: 투싼.csv
🚗 크롤링 중: 디 올 뉴 넥쏘 (4677)
✅ 저장 완료: 디 올 뉴 넥쏘.csv
🚗 크롤링 중: GV70 (4609)
✅ 저장 완료: GV70.csv
🚗 크롤링 중: Electrified GV70 (4705)
✅ 저장 완료: Electrified GV70.csv
🚗 크롤링 중: 산타페 (4435)
✅ 저장 완료: 산타페.csv
🚗 크롤링 중: 디 올 뉴 팰리세이드 (4699)
✅ 저장 완료: 디 올 뉴 팰리세이드.c

In [45]:
len(car_models)

34